In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 

import pickle

from tqdm import tqdm

import numpy as np

import pandas as pd

from sklearn.preprocessing import MinMaxScaler as mms

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from modules.utils.general_utils.utilities import group_wise_binning
from modules.utils.general_utils.visualizers import visualize_full_panel, visualize_temporal_panel

KeyboardInterrupt: 

In [ ]:
def sns_styleset():
    sns.set(context='paper', style='ticks', font='DejaVu Sans')
    matplotlib.rcParams['figure.dpi']        = 300
    matplotlib.rcParams['axes.linewidth']    = 1
    matplotlib.rcParams['xtick.major.width'] = 1
    matplotlib.rcParams['ytick.major.width'] = 1
    matplotlib.rcParams['xtick.major.size']  = 3
    matplotlib.rcParams['ytick.major.size']  = 3
    matplotlib.rcParams['xtick.minor.size']  = 2
    matplotlib.rcParams['ytick.minor.size']  = 2
    matplotlib.rcParams['font.size']         = 13
    matplotlib.rcParams['axes.titlesize']    = 13
    matplotlib.rcParams['axes.labelsize']    = 13
    matplotlib.rcParams['legend.fontsize']   = 13
    matplotlib.rcParams['xtick.labelsize']   = 13
    matplotlib.rcParams['ytick.labelsize']   = 13
    
sns_styleset()
cmap = matplotlib.cm.get_cmap('Paired')

## DEFINE  GLOBALS

In [ ]:
INPUTS_PATH = 'data\\train\\inputs\\{}'
TARGETS_PATH = 'data\\train\\targets\\{}'

CMAP = matplotlib.cm.get_cmap('Paired')

BTCH = [i for i in range(len(os.listdir(INPUTS_PATH.format('continuous_features'))))]
BTCH = BTCH[0::20]

SNAPSHOTS = 10

INPUTS = ['continuous_features', 'context']
TARGETS = [
    'tar_sessions',
    'tar_delta_sessions',    
    'tar_active_time',
    'tar_session_time',
    'tar_activity'
]

## LOAD

### 1 - Load Remappers

In [ ]:
MODELS_RMP = { 
    'melchior': 'RNN'
}
TARGETS_RMP = {
    'tar_delta_sessions': 'Future Absence',    
    'tar_active_time': 'Future Active Time',
    'tar_session_time': 'Future Session Time',
    'tar_activity': 'Future Session Activity',
    'tar_sessions': 'Future N° Sessions'
}
with open('results\\saved_objects\\mappers\\context.pkl', 'rb') as pickle_file:
    CONTEXT_RMP = pickle.load(pickle_file) 
CONTEXT_RMP = {value: key for key, value in CONTEXT_RMP.items()}

### 2 - Load Data Container

In [ ]:
with (open('results\\saved_data_containers\\melchior.pkl', 'rb')) as container:
    data_container = pickle.load(container)

# Plotting

In [ ]:
predictions = data_container['prediction_ds']
contexts = data_container['context']

### 1 - Full Panel Static

In [ ]:
for snapshot in range(10):
    
    UMAP = np.load(f'results\\saved_dim_reduction\\2D\\umap_melchior_eng_emb_{snapshot}.npy')
    UMAP = UMAP[~np.isnan(UMAP).any(axis=1)]
    colors_dict = {target_name: None for target_name in TARGETS}

    for color_name, array in colors_dict.items():

        binned_array = group_wise_binning(
            array=predictions[color_name][snapshot],
            grouper=contexts[snapshot],
            n_bins=100,
            method='discret'
        )
        colors_dict[color_name] = binned_array
        
    visualize_full_panel(
        reduction=UMAP, 
        contexts=contexts[snapshot],
        context_remap=CONTEXT_RMP,
        colors_dict=colors_dict, 
        colors_remap=TARGETS_RMP,
        colors_name=TARGETS, 
        cmapper=CMAP,
        snapshot=snapshot+1,
        save_name=f'umap_snapshot_{snapshot}',
        s=0.1
    )

### 2 - Full Panel Temporal

In [ ]:
for target in TARGETS:
    
    visualize_temporal_panel(
        temporal_color=predictions[target], 
        temporal_contexts=contexts, 
        color_name=TARGETS_RMP[target],
        binning_method='discret',
        context_remap=CONTEXT_RMP,
        cmapper=CMAP,
        s=0.05
    )

### 3 - PCA REDUCTION

In [ ]:
contexts

In [ ]:
for snapshot in range(10):
    
    PCA = np.load(f'results\\saved_dim_reduction\\2D\\pca_melchior_eng_emb_{snapshot}_cont_aware.npy')
    PCA = PCA[~np.isnan(PCA).any(axis=1)]
    colors_dict = {target_name: None for target_name in TARGETS}

    for color_name, array in colors_dict.items():

        binned_array = group_wise_binning(
            array=predictions[color_name][snapshot],
            grouper=contexts[snapshot],
            n_bins=100,
            method='discret'
        )
        colors_dict[color_name] = binned_array
    
    for unique_context in np.unique(contexts[snapshot]):
    
        fig = plt.figure(figsize=(15, 6), constrained_layout=True)
        spec = fig.add_gridspec(ncols=10, nrows=2)

        ax_context = fig.add_subplot(spec[:, :4])
        axs_metrics = [
            fig.add_subplot(spec[0, 4:6]),
            fig.add_subplot(spec[0, 6:8]),
            fig.add_subplot(spec[0, 8:]),
            fig.add_subplot(spec[1, 5:7]),
            fig.add_subplot(spec[1, 7:9])
        ]

        label = CONTEXT_RMP[int(unique_context)]
        context_index = np.argwhere(contexts[snapshot] == unique_context).flatten()
        c = CMAP(int(unique_context))
        ax_context.scatter(
            PCA[:, 0][context_index],
            PCA[:, 1][context_index],
            marker='o',
            edgecolor='',
            color=c,
            label=f'Object {label}',
            s=0.1
        )
        ax_context.set_ylabel('')
        ax_context.set_xlabel('')
        ax_context.set_title(f'Game Context {label} - $t$ {snapshot}')

        index = 0
        for color_name, ax_metric in zip(TARGETS, axs_metrics):

            img = ax_metric.scatter(
                PCA[:, 0][context_index],
                PCA[:, 1][context_index],
                c=colors_dict[color_name][context_index],
                marker='o',
                edgecolor='',
                cmap='coolwarm',
                vmin=0,
                vmax=100,
                s=0.1
            )
            ax_metric.set_title(
                f'{TARGETS_RMP[color_name]} - $t$ {snapshot}'
            )
            if index > 2:
                ax_metric.set_yticks([])
                ax_metric.set_xlabel('')
            else:
                ax_metric.set_yticks([])
                ax_metric.set_xticks([])
            index += 1

        fig.text(0.5, -0.01, 'Dimension 1', ha='center')
        fig.text(-0.01, 0.5, 'Dimension 2', va='center', rotation='vertical')
        handles, labels = ax_context.get_legend_handles_labels()
        leg = ax_context.legend(
            handles,
            labels,
            markerscale=8,
            ncol=1
        )
        leg.get_frame().set_edgecolor('k')

        plt.tight_layout()

        cbaxes = fig.add_axes([1.0, 0.1, 0.02, 0.8])
        cbar = fig.colorbar(
            img,
            cax=cbaxes,
            cmap='coolwarm',
            boundaries=np.linspace(0, 100, 100),
            ticks=[0, 25, 50, 75, 100]
        )
        cbar.set_label('Discretized Metric Value')

        #     plt.savefig(
        #         f'{save_path}\\{save_name}.png',
        #         dpi=500,
        #         bbox_inches='tight'
        #     )

        plt.show()

## ANIMATED 3D TEMPORAL REPPRESENTATION

In [ ]:
gif_index = 0
for snapshot in range(5):
    
    UMAP = np.load(f'results\\saved_dim_reduction\\3D\\umap_melchior_eng_emb_{snapshot}.npy')
    UMAP = UMAP[~np.isnan(UMAP).any(axis=1)]
    colors_dict = {target_name: None for target_name in TARGETS}
    
    fig = plt.figure(figsize=(20, 8))
    spec = fig.add_gridspec(ncols=5, nrows=2)
    
    ax_context = fig.add_subplot(spec[:, :2], projection='3d')
    ax_context.scatter(
        UMAP[:, 0], 
        UMAP[:, 1], 
        UMAP[:, 2],
        s=0.05,
        c=contexts[snapshot],
        marker='o',
        cmap='Paired'
    )
    ax_context.set_title(f'Game Context - $t$ {snapshot+1}')
    ax_context.axis('off')
    ax_context.set_xticks([])
    ax_context.set_yticks([])
    ax_context.set_zticks([])
    
    index = 0
    for column in range(2, 5):

        for row in range(2):

            target_name = TARGETS[index]
            ax_metric = fig.add_subplot(spec[row, column], projection='3d')
        
            binned_array = group_wise_binning(
                array=predictions[target_name][snapshot],
                grouper=contexts[snapshot],
                n_bins=100,
                method='discret'
            )

            ax_metric.scatter(
                UMAP[:, 0], 
                UMAP[:, 1], 
                UMAP[:, 2],
                s=0.05,
                c=binned_array,
                marker='o',
                cmap='coolwarm'
            )

            ax_metric.axis('off')
            ax_metric.set_xticks([])
            ax_metric.set_yticks([])
            ax_metric.set_zticks([])
            ax_metric.set_title(f'{TARGETS_RMP[target_name]} - $t$ {snapshot+1}')

            index+=1
     
    for angle in range(0, 360, 10):

        for ax in fig.axes:

            ax.view_init(0, angle)

        plt.savefig(f'results\\figures\\embeddings\\gifs\\{gif_index}.png')
        gif_index += 1
        print(gif_index)